In [2]:
import requests						# 发起网络请求
from bs4 import BeautifulSoup		# 解析HTML文本
import pandas as pd					# 处理数据
import os
import time			# 处理时间戳
import json			# 用来解析json文本

In [18]:
'''
用于发起网络请求
url : Request Url
kw  : Keyword
page: Page number
'''
def fetchUrl(url, kw, page):
    # 请求头
    headers={
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/json;charset=UTF-8",
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",
        "Referer": "https://blog.csdn.net/wenxuhonghe/article/details/113730696",
        "Accept-Encoding" : "gzip, deflate",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
        "Connection": "keep-alive",
        "Cookie": "__jsluid_h=feed647f64ed868f713978e151ffee30; sso_c=0; sfr=1",
        "Host": "search.people.cn",
        "Origin": "http://search.people.cn",
        "Referer": "http://search.people.cn/s/?keyword=%E6%96%87%E5%8C%96&st=0&_=1640775018893",
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62"
    }
    # 请求参数
    payloads = {
        "endTime": 0,
        "hasContent": True,
        "hasTitle": True,
        "isFuzzy": True,
        "key": kw,
        "limit": 10,
        "page": page,
        "sortType": 2,
        "startTime": 0,
        "type": 0,
    }

    # 发起 post 请求
    r = requests.post(url, headers=headers, data=json.dumps(payloads))
    return r.json()

In [13]:
def parseJson(jsonObj):
    #解析数据
    records = jsonObj["data"]["records"]
    for item in records:
        # 这里示例解析了几条，其他数据项如末尾所示，有需要自行解析
        pid = item["id"]
        originalName = item["originalName"]
        belongsName = item["belongsName"]
        content = BeautifulSoup(item["content"], "html.parser").text
        displayTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(item["displayTime"]/1000))
        subtitle = item["subtitle"]
        title = BeautifulSoup(item["title"], "html.parser").text
        url = item["url"]
        
        yield [[pid, title, subtitle, displayTime, originalName, belongsName, content, url]]

In [14]:
'''
用于将数据保存成 csv 格式的文件（以追加的模式）
path   : 保存的路径，若文件夹不存在，则自动创建
filename: 保存的文件名
data   : 保存的数据内容
'''
def saveFile(path, filename, data):
    # 如果路径不存在，就创建路径
    if not os.path.exists(path):
        os.makedirs(path)
    # 保存数据
    dataframe = pd.DataFrame(data)
    dataframe.to_csv(path + filename + ".csv", encoding='utf_8_sig', mode='a', index=False, sep=',', header=False )

In [22]:
if __name__ == "__main__":
	# 起始页，终止页，关键词设置
    start = 1
    end = 2000
    kw = "文化"
    # 保存表头行
    headline = [["文章id", "标题", "副标题", "发表时间", "来源", "版面", "摘要", "链接"]]
    saveFile("data/", kw, headline)
    #爬取数据
    for page in range(start, end + 1):
        url = "http://search.people.cn/search-platform/front/search"
        html = fetchUrl(url, kw, page)
        for data in parseJson(html):
            saveFile("data/", kw, data)
        print("第{}页爬取完成".format(page))
        time.sleep(2)
    
    # 爬虫完成提示信息
    print("爬虫执行完毕！数据已保存至以下路径中，请查看！")
    print(os.getcwd(), "\\data")

第1页爬取完成


KeyboardInterrupt: 

In [16]:
import pandas as pd
data = pd.read_csv('data/文化.csv', encoding='utf-8', usecols=[1,3,5,6])
data.drop_duplicates(subset='标题', inplace=True)
pd.set_option('display.max_colwidth', 100)
data.reset_index().drop('index', axis=1)

,标题,发表时间,版面,摘要
0,品味传统文化 喜迎中秋佳节,2021/9/21 6:57,#特别关注#,做月饼、猜灯谜、赏晚会……中秋佳节来临，各地举办精彩纷呈的文化活动，人们在游览体验中感受传统节日的文化魅力，乐享美好假期。北京市石景山区五里坨街道新时代文明实践所文化广场举办中秋节游园会，吸引...
1,信仰的力量：做红色文化的“追星人”,2021/8/7 16:15,#滚动#本地#教育#广东频道#,做红色文化的追星者 在众多“追星”团队中，有一支名叫“灯塔筑梦”的特殊队伍，他们既不是根据学院、专业构成的实践队伍，又不是按照特定条件招募组建的，团队的队员有一个共同点——都来自“青马灯...
2,版权保护让民族民间文化大放异彩,2021/3/10 16:39,#采编人员信息统计#贵州频道#贵州文教#频道稿件编辑统计查询#顾兰云#,做好民族民间文化的版权保护工作是激活文化资产价值的有力手段。乡村振兴战略实施中，人们对精神文化的需求将会更大，通过实施版权保护和运用，不但可以进一步增加优秀文化资源的供给，还可以进一步提高民族...
3,打造“隆平小镇”历史文化街区,2021/1/20 18:18,#市州县域#,做好“农”字文章—— 打造“隆平小镇”历史文化街区 “芙蓉区聚焦农业优势，大有可为！”1月18日，谈起长沙市芙蓉区优势，长沙市政协委员、芙蓉区政协主席郑明津津乐道。通过调研，郑明在今年长沙...
4,忠县：《忠文化干部读本》教材编写协议签订,2021/3/25 4:02,#区县#,座谈会前，调研组一行到作为忠文化“忠于事业”教学点的忠州腐乳厂，实地查看忠州豆腐乳生产和销售情况，了解企业秉持忠文化精神，感受忠文化在企业发展中的魅力。 随后，重庆市委党校与忠县签订《忠...
...,...,...,...,...
7897,在行走中触摸历史 百年南滨文化探秘活动启动,2021/12/8 12:27,#要闻#,“百年南滨”文化深度探秘活动启动。重庆市文物保护志愿者服务总队供图 人民网重庆12月8日电 近日，“百年南滨”文化深度探秘活动在南岸区南滨文化产业园启动。在文保志愿者、重...
7898,第三届敖包相会文化旅游节在后旗阿古拉文化旅游主题小镇举行,2020/8/26 17:51,#通辽#,8月25日，第三届敖包相会文化旅游节在科左后旗阿古拉文化旅游主题小镇拉开帷幕。本次文化旅游节为期三天，由市政府主办，市文旅广电局、科左后旗人民政府承办。副市长张怡致辞并宣布第三届敖包...
7899,金华历史文化研究成果展示厅启用,2021/11/26 9:10,#浙江频道#图片#今日热点#,11月25日，金华历史文化研究成果展示厅启用第一天，就吸引了不少市民前往参观，还有研究者来此查阅文献资料。展示厅内，金华历史文化之脉徐徐展开，述说着金华长期以来历史文化研究取得的丰硕成果。
7900,四川省第三届残疾人文化艺术节延期举办,2021/10/29 8:43,#社会#四川频道#,10月28日，封面新闻记者从四川省第三届残疾人文化艺术节组委会获悉，为切实做好当前疫情防控工作，原定于11月15日至19日举办的四川省第三届残疾人文化艺术节，延期在南充市举办，具体...


In [75]:

from spacy import displacy

import zh_core_web_sm

nlp = zh_core_web_sm.load()


def main():
    doc = nlp("王小明在北京的清华大学读书")
    for token in doc:
        print(token.text,token.ent_iob_, token.ent_type_
              )

    # displacy.serve(doc)


if __name__ == "__main__":
    main()

王小明 B PERSON
在 O 
北京 B GPE
的 O 
清华 B ORG
大学 I ORG
读书 O 


In [17]:
data = pd.concat([data, pd.DataFrame(columns=['DATE','ORG','PERSON','GPE'])])

In [18]:

data = data.fillna('')

In [9]:
data

,标题,发表时间,版面,摘要,DATE,ORG,PERSON,GPE
0,品味传统文化 喜迎中秋佳节,2021/9/21 6:57,#特别关注#,做月饼、猜灯谜、赏晚会……中秋佳节来临，各地举办精彩纷呈的文化活动，人们在游览体验中感受传统节日的文化魅力，乐享美好假期。北京市石景山区五里坨街道新时代文明实践所文化广场举办中秋节游园会，吸引...,,,,
4,信仰的力量：做红色文化的“追星人”,2021/8/7 16:15,#滚动#本地#教育#广东频道#,做红色文化的追星者 在众多“追星”团队中，有一支名叫“灯塔筑梦”的特殊队伍，他们既不是根据学院、专业构成的实践队伍，又不是按照特定条件招募组建的，团队的队员有一个共同点——都来自“青马灯...,,,,
5,版权保护让民族民间文化大放异彩,2021/3/10 16:39,#采编人员信息统计#贵州频道#贵州文教#频道稿件编辑统计查询#顾兰云#,做好民族民间文化的版权保护工作是激活文化资产价值的有力手段。乡村振兴战略实施中，人们对精神文化的需求将会更大，通过实施版权保护和运用，不但可以进一步增加优秀文化资源的供给，还可以进一步提高民族...,,,,
6,打造“隆平小镇”历史文化街区,2021/1/20 18:18,#市州县域#,做好“农”字文章—— 打造“隆平小镇”历史文化街区 “芙蓉区聚焦农业优势，大有可为！”1月18日，谈起长沙市芙蓉区优势，长沙市政协委员、芙蓉区政协主席郑明津津乐道。通过调研，郑明在今年长沙...,,,,
7,忠县：《忠文化干部读本》教材编写协议签订,2021/3/25 4:02,#区县#,座谈会前，调研组一行到作为忠文化“忠于事业”教学点的忠州腐乳厂，实地查看忠州豆腐乳生产和销售情况，了解企业秉持忠文化精神，感受忠文化在企业发展中的魅力。 随后，重庆市委党校与忠县签订《忠...,,,,
...,...,...,...,...,...,...,...,...
18956,在行走中触摸历史 百年南滨文化探秘活动启动,2021/12/8 12:27,#要闻#,“百年南滨”文化深度探秘活动启动。重庆市文物保护志愿者服务总队供图 人民网重庆12月8日电 近日，“百年南滨”文化深度探秘活动在南岸区南滨文化产业园启动。在文保志愿者、重...,,,,
18957,第三届敖包相会文化旅游节在后旗阿古拉文化旅游主题小镇举行,2020/8/26 17:51,#通辽#,8月25日，第三届敖包相会文化旅游节在科左后旗阿古拉文化旅游主题小镇拉开帷幕。本次文化旅游节为期三天，由市政府主办，市文旅广电局、科左后旗人民政府承办。副市长张怡致辞并宣布第三届敖包...,,,,
18958,金华历史文化研究成果展示厅启用,2021/11/26 9:10,#浙江频道#图片#今日热点#,11月25日，金华历史文化研究成果展示厅启用第一天，就吸引了不少市民前往参观，还有研究者来此查阅文献资料。展示厅内，金华历史文化之脉徐徐展开，述说着金华长期以来历史文化研究取得的丰硕成果。,,,,
18959,四川省第三届残疾人文化艺术节延期举办,2021/10/29 8:43,#社会#四川频道#,10月28日，封面新闻记者从四川省第三届残疾人文化艺术节组委会获悉，为切实做好当前疫情防控工作，原定于11月15日至19日举办的四川省第三届残疾人文化艺术节，延期在南充市举办，具体...,,,,


In [127]:
from tqdm import tqdm
for i,text in tqdm(enumerate(data['摘要'])):
    doc = nlp(text)
    for token in doc:
        # temp = ''
        if token.ent_iob_=='B':
            if token.ent_type_=='DATE':
                data.iloc[i,4] = data.iloc[i,4]+';'+token.text
            elif token.ent_type_ =='ORG':
                data.iloc[i,5] = data.iloc[i,5]+';'+token.text
            elif token.ent_type_ =='PERSON':
                data.iloc[i,6] = data.iloc[i,6]+';'+token.text
            elif token.ent_type_ =='GPE':
                data.iloc[i,7] = data.iloc[i,7]+';'+token.text
        elif token.ent_iob_=='I':
            if token.ent_type_=='DATE':
                data.iloc[i,4] = data.iloc[i,4]+token.text
            elif token.ent_type_ =='ORG':
                data.iloc[i,5] = data.iloc[i,5]+token.text
            elif token.ent_type_ =='PERSON':
                data.iloc[i,6] = data.iloc[i,6]+token.text
            elif token.ent_type_ =='GPE':
                data.iloc[i,7] = data.iloc[i,7]+token.text

7902it [02:34, 51.28it/s]


In [128]:
data

,标题,发表时间,版面,摘要,DATE,ORG,PERSON,GPE
0,品味传统文化 喜迎中秋佳节,2021/9/21 6:57,#特别关注#,做月饼、猜灯谜、赏晚会……中秋佳节来临，各地举办精彩纷呈的文化活动，人们在游览体验中感受传统节日的文化魅力，乐享美好假期。北京市石景山区五里坨街道新时代文明实践所文化广场举办中秋节游园会，吸引...,;五号,;北京市石景山区;江西省博物馆;中国航天;云南分公司,;赏晚会,;天南地北
4,信仰的力量：做红色文化的“追星人”,2021/8/7 16:15,#滚动#本地#教育#广东频道#,做红色文化的追星者 在众多“追星”团队中，有一支名叫“灯塔筑梦”的特殊队伍，他们既不是根据学院、专业构成的实践队伍，又不是按照特定条件招募组建的，团队的队员有一个共同点——都来自“青马灯...,,;广东医青年;广东医科大学;广东医青年,;徐畅;造星;栏目,;广东省
5,版权保护让民族民间文化大放异彩,2021/3/10 16:39,#采编人员信息统计#贵州频道#贵州文教#频道稿件编辑统计查询#顾兰云#,做好民族民间文化的版权保护工作是激活文化资产价值的有力手段。乡村振兴战略实施中，人们对精神文化的需求将会更大，通过实施版权保护和运用，不但可以进一步增加优秀文化资源的供给，还可以进一步提高民族...,,;彰显贵州文化特色和人文,,;贵州;贵州
6,打造“隆平小镇”历史文化街区,2021/1/20 18:18,#市州县域#,做好“农”字文章—— 打造“隆平小镇”历史文化街区 “芙蓉区聚焦农业优势，大有可为！”1月18日，谈起长沙市芙蓉区优势，长沙市政协委员、芙蓉区政协主席郑明津津乐道。通过调研，郑明在今年长沙...,;1月18日;郑明;今年;郑明,;长沙市;芙蓉区政协,;郑明津;郑明;陈新,;长沙
7,忠县：《忠文化干部读本》教材编写协议签订,2021/3/25 4:02,#区县#,座谈会前，调研组一行到作为忠文化“忠于事业”教学点的忠州腐乳厂，实地查看忠州豆腐乳生产和销售情况，了解企业秉持忠文化精神，感受忠文化在企业发展中的魅力。 随后，重庆市委党校与忠县签订《忠...,,;重庆市委党校;丰富忠,,
...,...,...,...,...,...,...,...,...
18956,在行走中触摸历史 百年南滨文化探秘活动启动,2021/12/8 12:27,#要闻#,“百年南滨”文化深度探秘活动启动。重庆市文物保护志愿者服务总队供图 人民网重庆12月8日电 近日，“百年南滨”文化深度探秘活动在南岸区南滨文化产业园启动。在文保志愿者、重...,;百年;12月8日;百年;百年;第一个周末;百年,;重庆市文物;重庆市导游协会;重庆市文物,,;重庆;重庆;南滨路
18957,第三届敖包相会文化旅游节在后旗阿古拉文化旅游主题小镇举行,2020/8/26 17:51,#通辽#,8月25日，第三届敖包相会文化旅游节在科左后旗阿古拉文化旅游主题小镇拉开帷幕。本次文化旅游节为期三天，由市政府主办，市文旅广电局、科左后旗人民政府承办。副市长张怡致辞并宣布第三届敖包...,;25日;三天,;市政府;中国艺术摄影学会;科尔沁美食品鉴会,;张怡致,
18958,金华历史文化研究成果展示厅启用,2021/11/26 9:10,#浙江频道#图片#今日热点#,11月25日，金华历史文化研究成果展示厅启用第一天，就吸引了不少市民前往参观，还有研究者来此查阅文献资料。展示厅内，金华历史文化之脉徐徐展开，述说着金华长期以来历史文化研究取得的丰硕成果。,;11月25日;第一天,;金华历史文化研究成果,,
18959,四川省第三届残疾人文化艺术节延期举办,2021/10/29 8:43,#社会#四川频道#,10月28日，封面新闻记者从四川省第三届残疾人文化艺术节组委会获悉，为切实做好当前疫情防控工作，原定于11月15日至19日举办的四川省第三届残疾人文化艺术节，延期在南充市举办，具体...,;11月15日至19日,,,;四川省


In [129]:
data.to_csv('pro1.csv')

In [111]:
data1 = pd.read_csv('1.csv')

In [114]:
data1 = data1.drop('Column1', axis=1)

In [19]:
import spacy
nlp = spacy.load('zh_core_web_lg')

In [51]:
from tqdm import tqdm
type_list = ['DATE', 'ORG', 'PERSON','GPE']
for j,text in tqdm(enumerate(data['摘要'])):
    ent_dict ={'DATE':[],'ORG':[],'PERSON':[],'GPE':[]}
    doc = nlp(text)
    for token in doc.ents:
        if token.label_ in type_list:
            idx = type_list.index(token.label_)
            ent_dict[token.label_].append(token.text)
            for i in type_list:
                ent_dict[i]=list(set(ent_dict[i]))
    data.iloc[j,4]=';'.join(ent_dict['DATE'])
    data.iloc[j,5]=';'.join(ent_dict['ORG'])
    data.iloc[j,6]=';'.join(ent_dict['PERSON'])
    data.iloc[j,7]=';'.join(ent_dict['GPE'])

7902it [02:35, 50.79it/s]


In [54]:
data.to_csv('culture.csv')

In [58]:
data.reset_index().drop(['版面','index'], axis=1).to_csv('culture.csv')

In [60]:
data.reset_index().drop(['版面','index'], axis=1)

,标题,发表时间,摘要,DATE,ORG,PERSON,GPE
0,品味传统文化 喜迎中秋佳节,2021/9/21 6:57,做月饼、猜灯谜、赏晚会……中秋佳节来临，各地举办精彩纷呈的文化活动，人们在游览体验中感受传统节日的文化魅力，乐享美好假期。北京市石景山区五里坨街道新时代文明实践所文化广场举办中秋节游园会，吸引...,五号;中秋;中秋节,江西省博物馆;中建二局;内蒙古博物院线上,,北京市;石景山区;云南;中国
1,信仰的力量：做红色文化的“追星人”,2021/8/7 16:15,做红色文化的追星者 在众多“追星”团队中，有一支名叫“灯塔筑梦”的特殊队伍，他们既不是根据学院、专业构成的实践队伍，又不是按照特定条件招募组建的，团队的队员有一个共同点——都来自“青马灯...,,广东医科大学,徐畅;中诉,广东省;广东
2,版权保护让民族民间文化大放异彩,2021/3/10 16:39,做好民族民间文化的版权保护工作是激活文化资产价值的有力手段。乡村振兴战略实施中，人们对精神文化的需求将会更大，通过实施版权保护和运用，不但可以进一步增加优秀文化资源的供给，还可以进一步提高民族...,,,,贵州
3,打造“隆平小镇”历史文化街区,2021/1/20 18:18,做好“农”字文章—— 打造“隆平小镇”历史文化街区 “芙蓉区聚焦农业优势，大有可为！”1月18日，谈起长沙市芙蓉区优势，长沙市政协委员、芙蓉区政协主席郑明津津乐道。通过调研，郑明在今年长沙...,1月18日;今年,芙蓉区政协;长沙市政协;驻街单位;农科院,郑明津;陈新;郑明,长沙;长沙市
4,忠县：《忠文化干部读本》教材编写协议签订,2021/3/25 4:02,座谈会前，调研组一行到作为忠文化“忠于事业”教学点的忠州腐乳厂，实地查看忠州豆腐乳生产和销售情况，了解企业秉持忠文化精神，感受忠文化在企业发展中的魅力。 随后，重庆市委党校与忠县签订《忠...,,市委党校;忠县忠文化干部学院;重庆市委党校,弘扬忠,忠州;忠县
...,...,...,...,...,...,...,...
7897,在行走中触摸历史 百年南滨文化探秘活动启动,2021/12/8 12:27,“百年南滨”文化深度探秘活动启动。重庆市文物保护志愿者服务总队供图 人民网重庆12月8日电 近日，“百年南滨”文化深度探秘活动在南岸区南滨文化产业园启动。在文保志愿者、重...,第一个周末;12月8日;百年,重庆金牌导游童思斯;重庆市导游协会;重庆市文物保护志愿者服务总队;人民网,,重庆
7898,第三届敖包相会文化旅游节在后旗阿古拉文化旅游主题小镇举行,2020/8/26 17:51,8月25日，第三届敖包相会文化旅游节在科左后旗阿古拉文化旅游主题小镇拉开帷幕。本次文化旅游节为期三天，由市政府主办，市文旅广电局、科左后旗人民政府承办。副市长张怡致辞并宣布第三届敖包...,8月25日;三天,科尔沁美食品;市政府;中国艺术摄影学会,张怡致,科左;阿古拉
7899,金华历史文化研究成果展示厅启用,2021/11/26 9:10,11月25日，金华历史文化研究成果展示厅启用第一天，就吸引了不少市民前往参观，还有研究者来此查阅文献资料。展示厅内，金华历史文化之脉徐徐展开，述说着金华长期以来历史文化研究取得的丰硕成果。,第一天;11月25日,,,金华
7900,四川省第三届残疾人文化艺术节延期举办,2021/10/29 8:43,10月28日，封面新闻记者从四川省第三届残疾人文化艺术节组委会获悉，为切实做好当前疫情防控工作，原定于11月15日至19日举办的四川省第三届残疾人文化艺术节，延期在南充市举办，具体...,10月28日;11月15日至19日,,,四川省;南充市
